In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
from urllib.request import urlopen
from urllib.request import Request 
from urllib.parse import urlencode, quote_plus

import urllib.request as ul
import xmltodict
import json
import sys
import io

In [3]:
import requests

In [4]:
from datetime import datetime
start = pd.to_datetime('20130101')
end=pd.to_datetime('20190331')
df=pd.DataFrame(columns=['연월일'])
df['연월일']=pd.date_range(start, end,freq='D')

In [5]:
df

,연월일
0,2013-01-01
1,2013-01-02
2,2013-01-03
3,2013-01-04
4,2013-01-05
...,...
2276,2019-03-27
2277,2019-03-28
2278,2019-03-29
2279,2019-03-30


In [6]:
def make_locdate(x):
    Y, M, D = x.split('-')
    return Y+M+D

In [7]:
df['연월일']=df['연월일'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2281 entries, 0 to 2280
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연월일     2281 non-null   object
dtypes: object(1)
memory usage: 17.9+ KB


In [8]:
df['locdate'] = df['연월일'].apply(lambda x:make_locdate(x))
df

,연월일,locdate
0,2013-01-01,20130101
1,2013-01-02,20130102
2,2013-01-03,20130103
3,2013-01-04,20130104
4,2013-01-05,20130105
...,...,...
2276,2019-03-27,20190327
2277,2019-03-28,20190328
2278,2019-03-29,20190329
2279,2019-03-30,20190330


url = 'http://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService/getAreaRiseSetInfo'

params ={'serviceKey' : '0GE5dffPQ2w31OZOACOdSgFRQzPihrAaslHTL1WR8UajH2KJivTGQfGkmw8pCFaWIr9sG2Vh4YEuaRk4rvCgSg==', 'locdate' : '20220505', 'location' : '서울' }
response = requests.get(url, params)
resp = xmltodict.parse(response.text)
resp

In [ ]:
url = 'http://apis.data.go.kr/B090041/openapi/service/RiseSetInfoService/getAreaRiseSetInfo'

for i in range(0, df.shape[0]):
    locdate = df.loc[i,'locdate']
    params ={'serviceKey' : '개인정보모자이크==', 'locdate' : locdate, 'location' : '서울' }
    response = requests.get(url, params)
    resp = xmltodict.parse(response.text)
    print(resp)
    df.loc[i, 'sunrise'] = resp['response']['body']['items']['item']['sunrise']
    df.loc[i, 'sunset'] = resp['response']['body']['items']['item']['sunset']
    print(i)
print('end')

In [ ]:
df

In [ ]:
df['sunset'] = df['sunset'].astype(int)//100
df['sunrise']= df['sunrise'].astype(int)//100

In [ ]:
df['일사']= df["sunset"] - df["sunrise"]

In [ ]:
df.rename(columns={'연월일':'date'}, inplace=True)

In [ ]:
df2 = df[['date','sunrise','sunset','일사']]
df2

### 1. train 데이터(2013-01-01 ~ 2018-12-31)

In [ ]:
start = df2['date']>='2013-01-01'
end = df2['date']<='2018-12-31'
train_일출일몰 = df2[start & end]
train_일출일몰

In [ ]:
train_일출일몰.to_csv('./make_data/0C_train_일출일몰_예보.csv', index=False)

### 2. test 데이터(2019-01-01 ~ 2019-03-31)

In [ ]:
start2 = df2['date']>='2019-01-01'
end2 = df2['date']<='2019-03-31'
test_일출일몰 = df2[start2 & end2]
test_일출일몰

In [ ]:
test_일출일몰.to_csv('./make_data/0C_test_일출일몰_예보.csv', index=False)